# Impoart

In [1]:
import import_ipynb

import tqdm
import torch
import pickle
import torchvision

import numpy as np
import bbox_visualizer as bbv
import matplotlib.pyplot as plt

import os

from pathlib import Path
from VOC2012_MaskRCNN_InstanceSegmentation import *
from torch.nn.parallel import DistributedDataParallel as DDP

In [2]:
def train_val(model, train_data_loader, val_data_loader, optimizer, scheduler, epoch = 1, device = "cpu",
    val_frequence = 1, print_frequency = None, step_save_frequency =  None, save_dir = ""):
    """
    [Operation]
        입력된 모델과 데이터에 대해 정해진 에폭만큼 train, val을 수행하고 결과 반환
    """
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    train_loss_list, val_loss_list = [], []

    for e in range(1, epoch+1):
        # train_loss 계산 및 학습
        train_loss = Task.train_one_epoch(model, train_data_loader, optimizer, scheduler, device)
        train_loss_list.append(train_loss)

        # val_frequence 마다 한 번씩 val_loss 계산
        val_loss = Task.loss_eval(model, device, val_data_loader) if (e % val_frequence == 0) else val_loss_list[-1]        
        val_loss_list.append(val_loss)

        # print_frequency마다 한번 씩 loss 출력
        if (print_frequency is not None) and (e % print_frequency == 0):
            print(f"epoch({e}) train_loss: {train_loss} val_loss: {val_loss}")

        if (step_save_frequency is not None) and (e%step_save_frequency == 0):
            torch.save(model.state_dict(), f"{save_dir}/step_model({e}-epoch)")

        
    loss_dict = {"train_loss_list": train_loss_list, "val_loss_list": val_loss_list}

    with open(f"{save_dir}/loss_dict.p", 'wb') as file:
        pickle.dump(loss_dict, file)


    torch.save(model.state_dict(), f"{save_dir}/final_model({epoch}-epoch)")



In [3]:
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'

    # 작업 그룹 초기화
    torch.distributed.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    torch.distributed.destroy_process_group()


def parallel(rank = 0, world_size=1):
    setup(rank, world_size)


    np.random.seed(0)

    device = rank if rank != None else "cpu"
    model = Task.get_model().to(rank)
    model = DDP(model, device_ids=[rank])
    t_dataset, t_dataloader, v_dataset, v_dataloader = Task.get_dataset_and_loader(10, rank=rank, world_size=world_size)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** epoch, last_epoch=-1, verbose=False)


    train_val(model, t_dataloader, v_dataloader, optimizer = optimizer, scheduler = scheduler,
            epoch=200, device = device,
        val_frequence = 1, print_frequency = 10, step_save_frequency = 10, save_dir = "result/test")
    
    cleanup()
    
parallel(rank = 0, world_size=1)


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
 24%|██▍       | 36/147 [03:42<10:34,  5.71s/it]

##### Train

In [3]:
from torchvision.models.detection import roi_heads
from torchvision.models.detection.roi_heads import * 

def boundary_mask(mask, filter_size = 1):
    n = filter_size**2
    conv = torch.nn.Conv2d(1, 1, (filter_size, filter_size), bias = False, padding="same").to(device)
    with torch.no_grad():
        conv.weight[0, 0] = 1
    mask = conv(mask)
    mask = torch.abs(mask - np.ceil(n/2)) < n/2-1
    return mask

def my_maskrcnn_loss(mask_logits, proposals, gt_masks, gt_labels, mask_matched_idxs): 
    # type: (Tensor, List[Tensor], List[Tensor], List[Tensor], List[Tensor]) 
    """ 
    Arguments: 
        proposals (list[BoxList]) 
        mask_logits (Tensor) 
        targets (list[BoxList]) 

    Return: 
        mask_loss (Tensor): scalar tensor containing the loss 
    """ 

    
    discretization_size = mask_logits.shape[-1] 
    labels = [l[idxs] for l, idxs in zip(gt_labels, mask_matched_idxs)] 
    mask_targets = [ 
        project_masks_on_boxes(m, p, i, discretization_size) 
        for m, p, i in zip(gt_masks, proposals, mask_matched_idxs) 
    ] 

    labels = torch.cat(labels, dim=0) 
    mask_targets = torch.cat(mask_targets, dim=0) 

    # torch.mean (in binary_cross_entropy_with_logits) doesn't 
    # accept empty tensors, so handle it separately 
    if mask_targets.numel() == 0: 
        return mask_logits.sum() * 0 

    # Weight
    if BOUNDARY_LOSS = True:
        mask_weights = [boundary_mask(mask_target.unsqueeze(0), BOUNDARY_SIZE).squeeze() for mask_target in mask_targets]
        mask_weights = 1+torch.stack(mask_weights)
    else:
        mask_weights = False
        
    mask_loss = F.binary_cross_entropy_with_logits( 
        mask_logits[torch.arange(labels.shape[0], device=labels.device), labels], mask_targets , weight=mask_weights
    ) 
    # mask_loss = F.binary_cross_entropy_with_logits( 
    #     mask_logits[torch.arange(labels.shape[0], device=labels.device), labels], mask_targets
    # ) 
    return mask_loss 

roi_heads.maskrcnn_loss = my_maskrcnn_loss

In [4]:
BOUNDARY_LOSS = True
BOUNDARY_SIZE = 1
np.random.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
t_dataset, t_dataloader, v_dataset, v_dataloader = Task.get_dataset_and_loader(10)
model = Task.get_model()

train_val(model, t_dataloader, v_dataloader, epoch=50, device = device, 
    val_frequence = 1, print_frequency = 10, step_save_frequency = 10, save_dir = "result/boundary_loss(size=1)")


/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 147/147 [01:13<00:00,  2.01it/s]


epoch(10) train_loss: 0.06045083701610565 val_loss: 0.061325836926698685


100%|██████████| 147/147 [01:13<00:00,  1.99it/s]


epoch(20) train_loss: 0.045523952692747116 val_loss: 0.046931490302085876


100%|██████████| 147/147 [01:12<00:00,  2.01it/s]


epoch(30) train_loss: 0.040491797029972076 val_loss: 0.041505053639411926


100%|██████████| 147/147 [01:12<00:00,  2.03it/s]


epoch(40) train_loss: 0.03834081441164017 val_loss: 0.038937631994485855


100%|██████████| 147/147 [01:13<00:00,  2.01it/s]


epoch(50) train_loss: 0.037238284945487976 val_loss: 0.03752163425087929


In [5]:
BOUNDARY_LOSS = True
BOUNDARY_SIZE = 2
np.random.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
t_dataset, t_dataloader, v_dataset, v_dataloader = Task.get_dataset_and_loader(10)
model = Task.get_model()

train_val(model, t_dataloader, v_dataloader, epoch=50, device = device, 
    val_frequence = 1, print_frequency = 10, step_save_frequency = 10, save_dir = "result/boundary_loss(size=2)")


  0%|          | 0/147 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 147/147 [01:15<00:00,  1.95it/s]


epoch(10) train_loss: 0.085898257791996 val_loss: 0.088671013712883


100%|██████████| 147/147 [01:13<00:00,  2.01it/s]


epoch(20) train_loss: 0.06755546480417252 val_loss: 0.07014336436986923


  0%|          | 0/147 [00:00<?, ?it/s]

epoch(30) train_loss: 0.06089160963892937 val_loss: 0.06226196885108948


  0%|          | 0/147 [00:00<?, ?it/s]

epoch(40) train_loss: 0.05814626067876816 val_loss: 0.05889860913157463


100%|██████████| 147/147 [01:15<00:00,  1.94it/s]


epoch(50) train_loss: 0.056724805384874344 val_loss: 0.05730310454964638


In [6]:
BOUNDARY_LOSS = True
BOUNDARY_SIZE = 3
np.random.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
t_dataset, t_dataloader, v_dataset, v_dataloader = Task.get_dataset_and_loader(10)
model = Task.get_model()

train_val(model, t_dataloader, v_dataloader, epoch=50, device = device, 
    val_frequence = 1, print_frequency = 10, step_save_frequency = 10, save_dir = "result/boundary_loss(size=3)")


  0%|          | 0/147 [00:00<?, ?it/s]

epoch(10) train_loss: 0.11076542735099792 val_loss: 0.1166163757443428


  0%|          | 0/147 [00:00<?, ?it/s]

epoch(20) train_loss: 0.08734365552663803 val_loss: 0.09161048382520676


100%|██████████| 147/147 [01:13<00:00,  1.99it/s]


epoch(30) train_loss: 0.07964629679918289 val_loss: 0.08195424824953079


  0%|          | 0/147 [00:00<?, ?it/s]

epoch(40) train_loss: 0.07627782970666885 val_loss: 0.07702791690826416


100%|██████████| 147/147 [01:13<00:00,  1.99it/s]


epoch(50) train_loss: 0.07457830011844635 val_loss: 0.07512140274047852
